In [78]:
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
import pandas as pd

custom_headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}

def get_product_info(url):
    r = requests.get(url, headers=custom_headers)
    soup = BeautifulSoup(r.text, 'lxml')
    
    names = soup.select_one('#productTitle')
    name = names.text.strip() if names else None
    
    ratings = soup.select_one('#acrPopover')
    rating_text = ratings.attrs.get('title') if ratings else None
    rating = rating_text.replace(' out of 5 stars', '') if rating_text else None

    prices = soup.select_one('span.a-price span.a-offscreen')
    price = prices.text.strip() if prices else None

    descriptions = soup.select_one('#feature-bullets')
    description = descriptions.text.strip() if descriptions else None

    # category_element = soup.select_one('span.a-color-state.a-text-bold')
    # category = category_element.text.strip() if category_element else None

    return {
        "name": name,
        "rating": rating,
        "price": price,
        "description": description
        # "category": category
    }

def parse_listings(category_urls, num_pages=1):
    all_page_data = []  # Store data from all pages
    for category_url in category_urls:
        for page_num in range(1, num_pages+1):
            url = f"{category_url}&page={page_num}"  # Append page number to URL
            r = requests.get(url, headers=custom_headers)
            soup_search = BeautifulSoup(r.text, 'lxml')
            link_elements = soup_search.select("[data-asin] h2 a")
            page_data = []
            for link in link_elements:
                full_url = urljoin(url, link.attrs.get("href"))
                product_info = get_product_info(full_url)
                page_data.append(product_info)
            all_page_data.extend(page_data)  # Add data from current page to all_page_data

    df = pd.DataFrame(all_page_data)
    df.to_csv("amazon_data.csv")

# Example usage with multiple category URLs and 3 pages per category:
category_urls = [
    "https://www.amazon.in/s?k=electronic+under+100000&crid=2WQCQASBTS8ON&sprefix=%2Caps%2C278&ref=nb_sb_ss_recent_2_0_recent"
    "https://www.amazon.in/s?k=headphones&rh=n%3A1389401031&ref=nb_sb_noss",
    "https://www.amazon.in/s?k=home+appliances&crid=31FEH6HOURKNA&sprefix=home+%2Caps%2C282&ref=nb_sb_ss_ts-doa-p_8_5"
]
parse_listings(category_urls, num_pages=3)



In [50]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("amazon_data.csv")

df

,Unnamed: 0,name,rating,price,description
0,0,Skullcandy Cassette Wireless On-Ear Headphone ...,3.9,"₹2,849.00",About this item Upto 22 Hours of Battery+ 1...
1,1,wigswin Women Men Wireless Single Bluetooth He...,NaN,₹899.00,About this item Clear Hands-free: Triple-mi...
2,2,Srhythm NC25 Wireless Headphones Bluetooth 5.3...,4.3,"₹7,199.00",About this item Digital Active Noise Cancel...
3,3,boAt Bassheads 225 Wired in Ear Earphone with ...,4.1,₹699.00,About this item Innovative housing design e...
4,4,pTron Tangentbeat in-Ear Bluetooth 5.0 Wireles...,3.8,₹599.00,About this item Ergonomic Magnetic Earbuds;...
...,...,...,...,...,...
176,176,Pigeon by Stovekraft Mio Nonstick Aluminium Co...,3.7,"₹1,099.00",About this item Material: Aluminium ; Size:...
177,177,Eureka Forbes Quick Clean DX Vacuum Cleaner wi...,4.0,"₹3,699.00",About this item Post Purchase virtual Demo ...
178,178,"AGARO Grand Plus 6 in 1 Hand Blender 1000W, 10...",4.4,"₹3,499.00","About this item With 1000 Watts power, the ..."
179,179,Wipro Vesta 360 Watt 4 in 1 Multicooker Egg Bo...,4.2,"₹1,700.00",About this item Four in one multicooker : M...


In [51]:
df.shape

(181, 5)

In [52]:
df.isnull().sum()

Unnamed: 0      0
name            0
rating         11
price           0
description     1
dtype: int64

In [53]:
df.dropna(inplace=True)  # Drop rows with NaN values

In [54]:
df.isnull().sum()

Unnamed: 0     0
name           0
rating         0
price          0
description    0
dtype: int64

In [55]:
# Check for duplicates
duplicate_rows = df.duplicated()

# Count the number of duplicates
num_duplicates = duplicate_rows.sum()

# Display the number of duplicates
print("Number of duplicate rows:", num_duplicates)

# Display the duplicate rows themselves (if any)
if num_duplicates > 0:
    print("Duplicate rows:")
    print(df[duplicate_rows])
else:
    print("No duplicate rows found.")


Number of duplicate rows: 0
No duplicate rows found.


In [56]:
df.shape

(169, 5)

In [57]:
# Convert the 'Product Price' column to string type
df['price'] = df['price'].astype(str).copy()

# Remove currency symbols and commas from the 'Product Price' column
df['price'] = df['price'].str.replace('₹', '').str.replace(',', '')

# Convert the 'Product Price' column to numeric values
df['price'] = pd.to_numeric(df['price'], errors='coerce')



In [58]:
df

,Unnamed: 0,name,rating,price,description
0,0,Skullcandy Cassette Wireless On-Ear Headphone ...,3.9,2849.0,About this item Upto 22 Hours of Battery+ 1...
2,2,Srhythm NC25 Wireless Headphones Bluetooth 5.3...,4.3,7199.0,About this item Digital Active Noise Cancel...
3,3,boAt Bassheads 225 Wired in Ear Earphone with ...,4.1,699.0,About this item Innovative housing design e...
4,4,pTron Tangentbeat in-Ear Bluetooth 5.0 Wireles...,3.8,599.0,About this item Ergonomic Magnetic Earbuds;...
5,5,Apple Wired EarPods with Lightning Connector,4.4,1799.0,"About this item Unlike traditional, circula..."
...,...,...,...,...,...
176,176,Pigeon by Stovekraft Mio Nonstick Aluminium Co...,3.7,1099.0,About this item Material: Aluminium ; Size:...
177,177,Eureka Forbes Quick Clean DX Vacuum Cleaner wi...,4.0,3699.0,About this item Post Purchase virtual Demo ...
178,178,"AGARO Grand Plus 6 in 1 Hand Blender 1000W, 10...",4.4,3499.0,"About this item With 1000 Watts power, the ..."
179,179,Wipro Vesta 360 Watt 4 in 1 Multicooker Egg Bo...,4.2,1700.0,About this item Four in one multicooker : M...


In [59]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df

,name,rating,price,description
0,Skullcandy Cassette Wireless On-Ear Headphone ...,3.9,2849.0,About this item Upto 22 Hours of Battery+ 1...
2,Srhythm NC25 Wireless Headphones Bluetooth 5.3...,4.3,7199.0,About this item Digital Active Noise Cancel...
3,boAt Bassheads 225 Wired in Ear Earphone with ...,4.1,699.0,About this item Innovative housing design e...
4,pTron Tangentbeat in-Ear Bluetooth 5.0 Wireles...,3.8,599.0,About this item Ergonomic Magnetic Earbuds;...
5,Apple Wired EarPods with Lightning Connector,4.4,1799.0,"About this item Unlike traditional, circula..."
...,...,...,...,...
176,Pigeon by Stovekraft Mio Nonstick Aluminium Co...,3.7,1099.0,About this item Material: Aluminium ; Size:...
177,Eureka Forbes Quick Clean DX Vacuum Cleaner wi...,4.0,3699.0,About this item Post Purchase virtual Demo ...
178,"AGARO Grand Plus 6 in 1 Hand Blender 1000W, 10...",4.4,3499.0,"About this item With 1000 Watts power, the ..."
179,Wipro Vesta 360 Watt 4 in 1 Multicooker Egg Bo...,4.2,1700.0,About this item Four in one multicooker : M...


In [60]:
df.to_csv("cleaned_dataa.csv", index=False)

In [68]:
import sqlite3
import pandas as pd

In [69]:
connection = sqlite3.connect("scraping.db")

In [70]:
print(connection.total_changes)

0


In [71]:
cursor = connection.cursor()

In [72]:
create_table_query = """
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY,
    name TEXT,
    rating REAL,
    price REAL,
    description TEXT
);
"""

In [73]:
# Execute the create table query
cursor.execute(create_table_query)

In [74]:
# Read the cleaned CSV file into a DataFrame
df = pd.read_csv("cleaned_dataa.csv")

# Create or connect to the SQLite database
conn = sqlite3.connect("scraping.db")

# Write the DataFrame to the "products" table in the database
df.to_sql("products", conn, if_exists="append", index=False)

# Commit the changes and close the connection
conn.commit()
conn.close()


In [75]:
cursor.execute("SELECT * FROM products")


In [76]:
rows = cursor.fetchall()
# Print the fetched rows
for row in rows:
    print(row)

(1, 'Skullcandy Cassette Wireless On-Ear Headphone with Mic (Black)', 3.9, 2849.0, 'About this item    Upto 22 Hours of Battery+ 10 Mins Rapid Charge for 2.5 Play Time    Headphone Type: Bluetooth On-Ear Connection Type: Bluetooth 5.0    Sound Pressure Level: 120-126dB    Impedance: 30 Ohms ±15%    2 Year Brand Warranty    Skullcandy Customer Care Number 0120-4341341 & Customer Care Email ID- customercare@brandeyes.in for customer assistance    \n\x9b  See more product details')
(2, 'Srhythm NC25 Wireless Headphones Bluetooth 5.3,Lightweight Noise Cancelling Headset Over-Ear with Low Latency,Game Mode', 4.3, 7199.0, "About this item    Digital Active Noise Cancelling Technology(ANC): Just press the ANC button, it will offer a pure immersive world for you whether in wired or Bluetooth mode. Professionally isolate 90% of surrounding noise from aircraft, subways,cars and crowds. (Kindly Note:It's NOT 100% noise cancellation. Human talking CAN NOT be elimated and ANC is less effective in q

In [77]:
# Connect to the SQLite database
conn = sqlite3.connect("scraping.db")

# Execute the SQL query and read the data into a DataFrame
df_from_db = pd.read_sql_query("SELECT * FROM products", conn)

# Display the DataFrame
print(df_from_db)

# Close the connection
conn.close()


      id                                               name  rating   price  \
0      1  Skullcandy Cassette Wireless On-Ear Headphone ...     3.9  2849.0   
1      2  Srhythm NC25 Wireless Headphones Bluetooth 5.3...     4.3  7199.0   
2      3  boAt Bassheads 225 Wired in Ear Earphone with ...     4.1   699.0   
3      4  pTron Tangentbeat in-Ear Bluetooth 5.0 Wireles...     3.8   599.0   
4      5       Apple Wired EarPods with Lightning Connector     4.4  1799.0   
..   ...                                                ...     ...     ...   
164  165  Pigeon by Stovekraft Mio Nonstick Aluminium Co...     3.7  1099.0   
165  166  Eureka Forbes Quick Clean DX Vacuum Cleaner wi...     4.0  3699.0   
166  167  AGARO Grand Plus 6 in 1 Hand Blender 1000W, 10...     4.4  3499.0   
167  168  Wipro Vesta 360 Watt 4 in 1 Multicooker Egg Bo...     4.2  1700.0   
168  169  Rico Japanese Technology Wireless Mini Electri...     3.9   899.0   

                                           descript